### Imports

In [ ]:
import numpy as np
import pandas as pd
import importlib
import os
import skimage.exposure
import matplotlib.pyplot as pyplot

import findatree.io as io
import findatree.interactive as interactive
import findatree.exporter as exporter

from bokeh.plotting import show, save
from bokeh.io import output_notebook, output_file

output_notebook()

### Define paths

In [ ]:
path_load = "/home/flostehr/data/lwf/processed/2020/hdf5/channels/tnr9996.hdf5" # Load channels & crowns from this path

path_dir_save_roi = "/home/flostehr/data/lwf/processed/2020/hdf5/rois/rgb/" # ROIs are saved to this directory
path_load_roi = "/home/flostehr/data/lwf/processed/2020/hdf5/rois/rgb/tnr9996_rois.hdf5"  #Load this ROI from hdf5

### Load channels & crowns hdf5

In [ ]:
importlib.reload(io)

data, params_data = io.load_hdf5(path_load)

channels, params_channels = data['channels'], params_data['channels']
crowns_human, params_crowns_human = data['crowns_human'], params_data['crowns_human']

del data, params_data

### Export ROIs to hdf5

In [ ]:
importlib.reload(exporter)

params_export = {
    'channels_export': ['red','green','blue'],
#     'channels_export': ['blue','green','red','re','nir','chm'],
    'query_export': 'id > -1',
    'size_export': 300,
}

rois, params_rois = exporter.export_rois(
    channels=channels,
    params_channels=params_channels,
    crowns=crowns_human,
    params_crowns=params_crowns_human,
    params_export=params_export,
    )

exporter.rois_to_hdf5(rois, params_rois, dir_name=path_dir_save_roi)

### Load ROIs from hdf5

In [ ]:
importlib.reload(exporter)

rois, params_rois = exporter.load_rois_from_hdf5(
    path_load_roi,
    load_sets=None, # This loads every set in rois group, set specifically with e.g. ['images', 'masks']
)


### Plot

In [ ]:
id = 1
i = np.argwhere(rois['ids']==id)[0][0]
img = rois['images'][:,:,:,i]
mask = rois['masks'][:,:,i]
img_masked = rois['images_masked'][:,:,:,i]

# Adjust exposure for better viewing
img = skimage.exposure.rescale_intensity(
    img, 
    in_range=(np.percentile(img ,0),np.percentile(img ,99.8)),
)
img_masked = skimage.exposure.rescale_intensity(
    img_masked, 
    in_range=(np.percentile(img_masked ,0),np.percentile(img_masked ,99.8)),
)

f = pyplot.figure(figsize=(12,6))
f.clear()
ax = f.add_subplot(131)
ax.imshow(img, interpolation='none', origin='lower')
ax.set_title('Padded image')

ax = f.add_subplot(132)
ax.imshow(img_masked, interpolation='none', origin='lower')
ax.set_title('Padded masked image')

ax = f.add_subplot(133)
ax.imshow(mask, interpolation='none', cmap='Greys_r', origin='lower')
_ = ax.set_title('Padded mask')

In [ ]:
importlib.reload(interactive)

plt = interactive.Plotter()
plt.width = 600
plt.channels_downscale = 2
plt.show_features = [
    'id', 'ba', 'kkl','nbv',
    'perc5_ndre','perc50_ndre'
]

plt.add_channels(channels, params_channels)

plt.figures_add_rgb(perc=2)

plt.togglers_add_crowns(crowns_human, params_crowns_human)

layout = plt.create_layout()

In [ ]:
show(layout)